1) Необходимо прочитать файл cinema.json, используя библиотеку json. https://docs.python.org/3/library/json.html

Создать новые файлы cinema2.json и cinema2.csv, в которых для каждого кинотеатра будут отображены только следующие поля из исходного файла: 

* ID, 
* Краткое наименование (CommonName), 
* Адрес (текстовой строкой, только улица-дом, без индекса и района), 
* Сайт, 
* Рабочие часы по дням недели (в формате словаря "День недели: часы работы"), 
* Долгота, 
* Широта.

In [9]:
import json
import csv

lst = []
with open("JSON/cinema.json", 'r') as json_file:
        lst.append(json.load(json_file))

lst2 = [{} for i in lst[0]]

with open('JSON/cinema2.csv', 'w', newline='') as csvfile:
    fieldnames = ['ID', 'CommonName', "Address", 'WebSite', 'WorkingHours', "latitude", 'longitude']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=';')
    writer.writeheader()
    
    for i, dic in enumerate(lst[0]):
        lst2[i]["ID"] = dic['global_id']
        lst2[i]["CommonName"] = dic['CommonName']
        lst2[i]["Address"] = dic['ObjectAddress'][0]['Address']
        lst2[i]['WebSite'] = dic['WebSite']
        lst2[i]['WorkingHours'] = {i['DayWeek']:i['WorkHours'] for i in dic['WorkingHours']} 
        lst2[i]["latitude"] = dic['geoData']['coordinates'][0][0]
        lst2[i]['longitude'] = dic['geoData']['coordinates'][0][1]

        writer.writerow({'ID': dic['global_id'],
                        'CommonName': dic['CommonName'],
                        "Address" : dic['ObjectAddress'][0]['Address'],
                        'WebSite': dic['WebSite'],
                        'WorkingHours': {i['DayWeek']:i['WorkHours'] for i in dic['WorkingHours']}, 
                        "latitude": dic['geoData']['coordinates'][0][0],
                        'longitude': dic['geoData']['coordinates'][0][1]})
        

    
with open('JSON/inst1.json', 'w') as f:
    json.dump(lst2, f)


2) Присутствуют два csv-файла. В одном ID и координаты узлов графа (points.csv), в другом - координаты ребер графа, полученные с географической карты (polylines.csv). Необходимо к координатам из ребер графа привязать соответствующие им узлы графа (смотреть по близости их координат) и создать csv файл с полями "ID начального узла ребра" и "ID конечного узла ребра". 

**Важно**: координаты узлов могут минимально расходиться с координатами начала и концов ребер, поэтому свести по точному соответствию не получится.

Справка по работе с csv: https://docs.python.org/3/library/csv.html

In [7]:
import json

lst9 = []

with open("JSON/cinema.json", 'r') as json_file:
        lst9.append(json.load(json_file))
print(lst9[0][14])

{'global_id': 477013008, 'TotalSeatsAmount': 120.0, 'CommonName': 'Кинотеатр «Юность»', 'FullName': 'Государственное бюджетное учреждение культуры города Москвы «Московское кино», Кинотеатр «Юность»', 'ShortName': 'ГБУК города Москвы «Московское кино», Кинотеатр «Юность»', 'OrgInfo': [{'OGRN': '1107746966029', 'ChiefName': 'Максимченко Светлана Борисовна', 'ChiefPhone': [{'ChiefPhone': '(495) 951-94-20'}], 'INN': '7719765428', 'KPP': '770501001', 'LegalAddress': '115035, г. Москва, Пятницкая улица, дом 2/38, строение 1\n', 'FullName': 'Государственное бюджетное учреждение культуры города Москвы «Московское кино»', 'ChiefPosition': 'генеральный директор'}], 'Category': 'Кинотеатр', 'ObjectAddress': [{'PostalCode': '123298', 'District': 'район Щукино', 'Address': 'улица Маршала Рыбалко, дом 1', 'Availability': [], 'AdmArea': 'Северо-Западный административный округ'}], 'ChiefOrg': 'Государственное бюджетное учреждение культуры города Москвы «Московское кино»', 'ChiefName': 'Максимченко Св

In [52]:
lst3 = []

with open('polylines.csv', 'r') as f:
    csv_reader = csv.reader(f, delimiter=';')
    for row in csv_reader:
        if row != []:
            lst3.append(row[0].replace('"', '').replace('),(', ');(').split(";"))

with open('polylines_remastered.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=';')
    
    for i in lst3:
        writer.writerow(i)

        
with open('polylines.csv', 'r') as f:
    csv_reader = csv.reader(f, delimiter=';')
    for row in csv_reader:
        if row != []:
            lst3.append(row[0].replace('"', '').replace('),(', ');(').split(";"))

with open('polylines_remastered.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=';')
    
    for i in lst3:
        writer.writerow(i)

[['(44.1652,56.1896)', '(44.04,56.2379)', '(44.0108,56.2366)', '(44.0053,56.2371)', '(44.0024,56.239)', '(44.0034,56.241)', '(44.0006,56.2399)', '(43.9941,56.2404)', '(43.9909,56.2407)', '(43.9818,56.2409)', '(43.9687,56.244)', '(43.9692,56.2455)', '(43.9735,56.2474)', '(43.9778,56.2455)', '(43.9779,56.2472)', '(43.9828,56.2465)', '(43.9854,56.2442)', '(43.9854,56.2452)', '(43.9864,56.2459)', '(43.9915,56.2443)'], ['(43.9865,56.2459)', '(43.9847,56.2491)'], ['(44.0054,56.2372)', '(44.0002,56.2338)'], ['(43.9734,56.2474)', '(43.9677,56.2493)', '(43.9685,56.2541)', '(43.9732,56.2596)'], ['(43.9821,56.2409)', '(43.9797,56.2373)', '(43.9837,56.2357)', '(43.9746,56.2355)', '(43.9752,56.2317)', '(43.9651,56.2344)', '(43.9656,56.235)', '(43.9601,56.2385)', '(43.955,56.2439)'], ['(43.9651,56.2344)', '(43.9571,56.2315)', '(43.9452,56.2325)', '(43.9396,56.2314)'], ['(43.9571,56.2316)', '(43.9512,56.2257)'], ['(43.9505,56.2257)', '(43.96,56.2158)', '(43.9716,56.2152)', '(44.0158,56.2114)'], ['(44

3) Создать json-файл, состоящий из ребер графа. Каждое ребро должно иметь уникальный ID (например, номер по порядку), к которому должны быть привязаны данные о начальном узле ("Source"), данные о конечном узле ребра ("Target"). Каждый узел в json должен содержать исходный id, long (долготу) и lat (широту).